In [1]:
#!pip3 install imutils

In [2]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [3]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [4]:
TRAINING_DIR = r"D:\Datasets\Dataset\train\train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = r"D:\Datasets\Dataset\test\test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [5]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [6]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10


C:\Users\harsh\AppData\Local\Temp\ipykernel_10640\3620006576.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


132/132 [==============================] - 36s 269ms/step - loss: 0.5326 - acc: 0.7529 - val_loss: 0.1897 - val_acc: 0.9536
Epoch 2/10
132/132 [==============================] - 66s 504ms/step - loss: 0.3254 - acc: 0.8783 - val_loss: 0.1113 - val_acc: 0.9536
Epoch 3/10
132/132 [==============================] - 67s 503ms/step - loss: 0.2587 - acc: 0.9042 - val_loss: 0.1264 - val_acc: 0.9536
Epoch 4/10
132/132 [==============================] - 66s 500ms/step - loss: 0.2430 - acc: 0.9019 - val_loss: 0.1410 - val_acc: 0.9381
Epoch 5/10
132/132 [==============================] - 70s 526ms/step - loss: 0.2380 - acc: 0.9057 - val_loss: 0.0551 - val_acc: 0.9794
Epoch 6/10
132/132 [==============================] - 69s 518ms/step - loss: 0.2175 - acc: 0.9171 - val_loss: 0.2003 - val_acc: 0.8969
Epoch 7/10
132/132 [==============================] - 68s 512ms/step - loss: 0.2050 - acc: 0.9270 - val_loss: 0.0587 - val_acc: 0.9742
Epoch 8/10
132/132 [==============================] - 70s 528ms/st

In [20]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model2-010.model")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()

<CascadeClassifier 00000214479E0650>
